In [1]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, StringIndexerModel
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Transformer

import mlflow
from mlflow.tracking import MlflowClient

In [2]:
import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'https://storage.yandexcloud.net'
os.environ['AWS_ACCESS_KEY_ID'] = '33kU43UzyCYfV1jgKUPL'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'WPZnfkNEOlpdZ32hwVGhQ6PNiPPjmFZEajnWUMRe'

In [3]:
mlflow.set_tracking_uri("https://mlflow.lab.karpov.courses")
mlflow.set_experiment(experiment_name = "PySpark-ML")

In [4]:
spark = SparkSession.builder.appName("PySparkTitanikJob")\
    .getOrCreate()

21/11/22 15:50:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
spark

In [6]:
mlflow.start_run()

<ActiveRun: >

In [7]:
titanic_df = spark.read.parquet('train.parquet')

In [8]:
titanic_df.show()

+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|Family_Size|Alone|
+--------+------+------+----+-----+-----+-------+--------+-----------+-----+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|          1|    0|
|       1|     1|female|38.0|    1|    0|71.2833|       C|          1|    0|
|       1|     3|female|26.0|    0|    0|  7.925|       S|          0|    1|
|       1|     1|female|35.0|    1|    0|   53.1|       S|          1|    0|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|          0|    1|
|       0|     3|  male|30.0|    0|    0| 8.4583|       Q|          0|    1|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|          0|    1|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|          4|    0|
|       1|     3|female|27.0|    0|    2|11.1333|       S|          2|    0|
|       1|     2|female|14.0|    1|    0|30.0708|       C|          1|    0|

In [9]:
train, test = titanic_df.randomSplit([0.8, 0.2])

In [10]:
indexer_sex = StringIndexer(inputCol="Sex", outputCol="Sex_index")

In [11]:
indexer_embarked = StringIndexer(inputCol="Embarked", outputCol="Embarked_index")

In [12]:
feature = VectorAssembler(
    inputCols=["Pclass","Age","SibSp","Parch","Fare","Family_Size","Embarked_index","Sex_index"],
    outputCol="features")

In [13]:
rf_classifier = RandomForestClassifier(labelCol="Survived", featuresCol="features")

In [14]:
pipeline = Pipeline(stages=[indexer_sex, indexer_embarked, feature, rf_classifier])

In [15]:
p_model = pipeline.fit(train)

In [16]:
prediction = p_model.transform(test)

In [17]:
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")

In [18]:
p_accuracy = evaluator_accuracy.evaluate(prediction)
p_f1 = evaluator_f1.evaluate(prediction)
print(f"Pipeline model [Accuracy] = {p_accuracy}")
print(f"Pipeline model [F1] = {p_f1}")

Pipeline model [Accuracy] = 0.8342541436464088
Pipeline model [F1] = 0.8319889038097847


In [19]:
for i in range(0, len(p_model.stages)):
    stage = p_model.stages[i]
    mlflow.log_param(f'stage_{i}_type', stage)
    if type(stage) is VectorAssembler:
        mlflow.log_param(f'stage_{i}_input', stage.getInputCols())
        mlflow.log_param(f'stage_{i}_output', stage.getOutputCol())
    elif type(stage) is StringIndexerModel:
        mlflow.log_param(f'stage_{i}_input', stage.getInputCol())
        mlflow.log_param(f'stage_{i}_output', stage.getOutputCol())
    else:
        mlflow.log_param(f'stage_{i}_features', stage.getFeaturesCol())
        mlflow.log_param(f'stage_{i}_label', stage.getLabelCol())

In [20]:
mlflow.log_param('MaxDepth', p_model.stages[-1].getMaxDepth())
mlflow.log_param('MaxNumTrees', p_model.stages[-1].getMaxBins())
mlflow.log_param('Impurity', p_model.stages[-1].getMinInfoGain())

In [21]:
mlflow.log_metric('accuracy', p_accuracy)
mlflow.log_metric('f1', p_f1)

In [22]:
mv = mlflow.spark.log_model(p_model,
                       artifact_path = "spark-model",
                       registered_model_name="spark-model")
mv

21/11/22 15:51:07 ERROR Instrumentation: org.apache.hadoop.fs.UnsupportedFileSystemException: No FileSystem for scheme "s3"
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3281)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3301)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:124)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3352)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3320)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:479)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:361)
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:673)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.super$save(Pipeline.scala:344)
	at org.apache.spark.ml.PipelineModel$PipelineModelWriter.$anonfun$save$4(Pipeline.scala:344)
	at org.apache.spark.ml.MLEvents.withSaveInstanceEvent(even

In [23]:
mlflow.end_run()